# Intro 
Supervised Learning

Date Range: Jan 2021 to June 2021

Linear regression
- Target    = Recovery cases
- Feature   = date, vaccination, case_new, case_active, case_unvax, case_fvax

KNN regression
- Target    = Recovery cases
- Feature   = date, vaccination, case_new, case_active, case_unvax, case_fvax

Other Techniques:
Linear Reg, KNN, decision tree, random forest, SVM, Neural Network

# Prerequisite

In [5]:
import random
import time
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# TensorFlow
import tensorflow as tf
from tensorflow.keras import layers, models

# Loading DataSet from Github

In [6]:
# URL should be https://raw.githubusercontent.com/username/repository/branch/filename.csv so no download needed

# Cases, Age, Location, more: https://github.com/MoH-Malaysia/covid19-public/tree/main/epidemic/{specific_file}
# Vaccination: https://github.com/CITF-Malaysia/citf-public/tree/main/vaccination/{specific_file}

url_cases = "https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/cases_malaysia.csv"
url_vax = "https://raw.githubusercontent.com/CITF-Malaysia/citf-public/main/vaccination/vax_malaysia.csv"


In [12]:
# Linear Regression
pd.set_option('display.max_columns', None)

df_cases = pd.read_csv(url_cases, usecols=['date', 'cases_new'])
df_vax = pd.read_csv(url_vax)

columns_to_read = ['date', 'cases_new']

start = '2021-09-01'
end = '2022-03-31'

df_cases = df_cases[(df_cases['date'] >= start) & (df_cases['date'] <= end)]
df_vax = df_vax[(df_vax['date'] >= start) & (df_vax['date'] <= end)]

print(df_cases.head())

           date  cases_new
585  2021-09-01      18762
586  2021-09-02      20988
587  2021-09-03      19378
588  2021-09-04      19057
589  2021-09-05      20396
